# Introduction
1. This script tunes hyperparameters for HER-DDPG

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
%matplotlib inline
from finrl import config
from finrl import config_tickers
from optuna.integration import PyTorchLightningPruningCallback

# Changed finrl_meta to meta
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
# from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy
# from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.agents.stablebaselines3.her import DRLHERAgent
# from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib
from finrl.meta.data_processor import DataProcessor
import joblib
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint

# Not needed for local run
# import sys
# sys.path.append("../FinRL-Library")

import itertools

c:\Users\frank\.conda\envs\m_env\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import torch
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")  # Set the device to GPU
else:
    print("GPU is not available. Using CPU.")
    device = torch.device("cpu")  # Set the device to CPU

GPU is available!


In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)  # datasets
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)  # trained_models
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)  # tensorboard_log
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)  # results

## Collecting data and preprocessing

In [5]:
#Custom ticker list dataframe download
# ticker_list = config_tickers.DOW_30_TICKER
# df = YahooDownloader(start_date = '2009-01-01',
#                      end_date = '2021-10-01',
#                      ticker_list = ticker_list).fetch_data()

df_list = []
TRAIN_START_DATE = '2010-01-01'
TEST_END_DATE = '2023-03-01'
for ticker in config_tickers.DOW_30_TICKER:
    # Fetch data for each ticker
    portfolio_raw_df = YahooDownloader(start_date=TRAIN_START_DATE,
                                       end_date=TEST_END_DATE,
                                       ticker_list=[ticker]).fetch_data()
    # Append the fetched DataFrame to the list
    df_list.append(portfolio_raw_df)

# Concatenate all DataFrames row-wise
df = pd.concat(df_list, axis=0, ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3311, 8)
Shape of DataFrame:  (3311, 8)


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3311, 8)


[*********************100%***********************]  1 of 1 completed


...

Shape of DataFrame:  (3311, 8)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (3311, 8)



[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (3311, 8)
Shape of DataFrame:  (994, 8)


In [6]:
print(df.head())

         date       open       high        low      close    volume  tic  day
0  2010-01-04  32.906181  40.919998  41.099998  40.810001   6894300  AXP    0
1  2010-01-05  32.833809  40.830002  41.230000  40.830002  10641200  AXP    1
2  2010-01-06  33.364555  41.490002  41.669998  41.230000   8399400  AXP    2
3  2010-01-07  33.905674  41.980000  42.240002  41.259998   8981700  AXP    3
4  2010-01-08  33.881458  41.950001  42.480000  41.759998   7907700  AXP    4


In [7]:
#You can add technical indicators and turbulence factor to dataframe
#Just set the use_technical_indicator=True, use_vix=True and use_turbulence=True
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators


Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [8]:
print(processed.head().to_markdown())

|    | date       |     open |     high |      low |   close |    volume | tic   |   day |   macd |   boll_ub |   boll_lb |   rsi_30 |   cci_30 |   dx_30 |   close_30_sma |   close_60_sma |   vix |   turbulence |
|---:|:-----------|---------:|---------:|---------:|--------:|----------:|:------|------:|-------:|----------:|----------:|---------:|---------:|--------:|---------------:|---------------:|------:|-------------:|
|  0 | 2010-01-04 |  6.44741 |  7.64321 |  7.66071 |  7.6225 | 493729600 | AAPL  |     0 |      0 |   7.70249 |    7.5843 |      100 |  66.6667 |     100 |         7.6225 |         7.6225 | 21.68 |            0 |
|  1 | 2010-01-04 | 40.5913  | 57.72    | 57.87    | 56.63   |   5277400 | AMGN  |     0 |      0 |   7.70249 |    7.5843 |      100 |  66.6667 |     100 |        56.63   |        56.63   | 21.68 |            0 |
|  2 | 2010-01-04 | 32.9062  | 40.92    | 41.1     | 40.81   |   6894300 | AXP   |     0 |      0 |   7.70249 |    7.5843 |      100 |  66.6667 |   

In [9]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full.sort_values(['date','tic'],ignore_index=True).head(5)

processed_full.to_csv('processed_full.csv')

In [10]:
train = data_split(processed_full, '2010-01-01','2021-10-01')
trade = data_split(processed_full, '2021-10-01','2023-03-01')
print(len(train))
print(len(trade))

85753
10237


In [11]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [12]:
#Defining the environment kwargs
num_stock_shares = [0] * stock_dimension
# From: https://github.com/AI4Finance-Foundation/FinRL/issues/540\
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,  # added argument
    "buy_cost_pct": buy_cost_list,  # changed to list
    "sell_cost_pct": sell_cost_list,  # changed to list
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}
#Instantiate the training gym compatible environment
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [13]:
#Instantiate the training environment
# Also instantiate our training gent
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))
agent = DRLHERAgent(env = env_train)

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [14]:
#Instantiate the trading environment
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, **env_kwargs)

## Tuning hyperparameters using Optuna

In [20]:
def sample_her_params(trial:optuna.Trial):
  # Size of the replay buffer
  buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
  batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
  
  return {"buffer_size": buffer_size,
          "learning_rate":learning_rate,
          "batch_size":batch_size}

In [16]:
from typing import Any
def sample_buffer_params(
    trial: optuna.Trial, hyperparams: dict[str, Any]
) -> dict[str, Any]:
    """
    Sampler for HerReplayBuffer hyperparams.

    :param trial:
    :parma hyperparams:
    :return:
    """
    her_kwargs = trial.her_kwargs.copy()
    her_kwargs["n_sampled_goal"] = trial.suggest_int("n_sampled_goal", 1, 5)
    her_kwargs["goal_selection_strategy"] = trial.suggest_categorical(
        "goal_selection_strategy", ["final", "episode", "future"]
    )
    her_kwargs["online_sampling"] = trial.suggest_categorical(
        "online_sampling", [True, False]
    )
    hyperparams["replay_buffer_kwargs"] = her_kwargs
    return hyperparams

In [17]:
#Calculate the Sharpe ratio
#This is our objective for tuning
def calculate_sharpe(df):
    df['daily_return'] = df['account_value'].pct_change(1)
    if df['daily_return'].std() != 0:
        sharpe = (252 ** 0.5) * df['daily_return'].mean()/ \
            df['daily_return'].std()
        return sharpe
    else:
        return 0

## Callbacks
1. The callback will terminate if the improvement margin is below certain point
2. It will terminate after certain number of trial_number are reached, not before that
3. It will hold its patience to reach the threshold

In [18]:
class LoggingCallback:
    def __init__(self, threshold: int, trial_number: int, patience: int):
        """
        threshold:int tolerance for increase in sharpe ratio
        trial_number: int Prune after minimum number of trials
        patience: int patience for the threshold
        """
        self.threshold = threshold
        self.trial_number = trial_number
        self.patience = patience
        self.cb_list = []  # Trials list for which threshold is reached

    def __call__(self, study: optuna.study, frozen_trial: optuna.Trial):
        # Setting the best value in the current trial
        study.set_user_attr("previous_best_value", study.best_value)

        # Checking if the minimum number of trials have pass
        if frozen_trial.number > self.trial_number:
            previous_best_value = study.user_attrs.get("previous_best_value", None)
            # Checking if the previous and current objective values have the same sign
            if previous_best_value * study.best_value >= 0:
                # Checking for the threshold condition
                if abs(previous_best_value - study.best_value) < self.threshold:
                    self.cb_list.append(frozen_trial.number)
                    # If threshold is achieved for the patience amount of time
                    if len(self.cb_list) > self.patience:
                        print("The study stops now...")
                        print(
                            "With number",
                            frozen_trial.number,
                            "and value ",
                            frozen_trial.value,
                        )
                        print(
                            "The previous and current best values are {} and {} respectively".format(
                                previous_best_value, study.best_value
                            )
                        )
                        study.stop()

HER with Optuna

In [23]:
from IPython.display import clear_output
import sys   

os.makedirs("HER_optuna_models",exist_ok=True)

def objective(trial:optuna.Trial):
  #Trial will suggest a set of hyperparamters from the specified range
  hyperparameters = sample_her_params(trial)
  model_her = agent.get_model(model_kwargs = hyperparameters )
  #You can increase it for better comparison
  trained_her = agent.train_model(model=model_her,
                                  tb_log_name="her" ,
                             total_timesteps=50000)
  trained_her.save('HER_optuna_models/her_{}.pth'.format(trial.number))
  clear_output(wait=True)
  #For the given hyperparamters, determine the account value in the trading period
  df_account_value, df_actions = DRLHERAgent.DRL_prediction(
    model=trained_her, 
    environment = e_trade_gym)
  #Calculate sharpe from the account value
  sharpe = calculate_sharpe(df_account_value)

  return sharpe

In [24]:
#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(study_name="her_study",direction='maximize',
                            sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=1e-5,patience=30,trial_number=5)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=30,catch=(ValueError,),callbacks=[logging_callback])

[I 2024-12-02 12:52:25,373] Trial 29 finished with value: -0.03299258355862492 and parameters: {'buffer_size': 100000, 'learning_rate': 0.020090753682926097, 'batch_size': 128}. Best is trial 20 with value: 0.2163387333834871.


hit end!


In [25]:
joblib.dump(study, "final_her_study__.pkl")

['final_her_study__.pkl']

In [26]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
print(f'Hyperparameters before tuning {{"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}}')

Hyperparameters after tuning {'buffer_size': 10000, 'learning_rate': 1.120018965138855e-05, 'batch_size': 128}
Hyperparameters before tuning {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


In [27]:
study.best_trial

FrozenTrial(number=20, state=TrialState.COMPLETE, values=[0.2163387333834871], datetime_start=datetime.datetime(2024, 12, 2, 10, 39, 21, 756403), datetime_complete=datetime.datetime(2024, 12, 2, 10, 52, 14, 82839), params={'buffer_size': 10000, 'learning_rate': 1.120018965138855e-05, 'batch_size': 128}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'buffer_size': CategoricalDistribution(choices=(10000, 100000, 1000000)), 'learning_rate': FloatDistribution(high=1.0, log=True, low=1e-05, step=None), 'batch_size': CategoricalDistribution(choices=(32, 64, 128, 256, 512))}, trial_id=20, value=None)

In [28]:
from stable_baselines3 import DDPG
tuned_model_her = DDPG.load('HER_optuna_models/her_{}.pth'.format(study.best_trial.number),env=env_train)


In [29]:
#Trading period account value with tuned model
df_account_value_tuned, df_actions_tuned = DRLHERAgent.DRL_prediction(
    model=tuned_model_her, 
    environment = e_trade_gym)

hit end!


In [30]:
print(df_account_value_tuned.head())
print(df_actions_tuned.head())

         date  account_value
0  2021-10-01   1.000000e+06
1  2021-10-04   1.001248e+06
2  2021-10-05   9.988362e+05
3  2021-10-06   9.978647e+05
4  2021-10-07   1.010576e+06
            AAPL  AMGN  AXP   BA  CAT  CRM  CSCO  CVX  DIS  GS  ...  MRK  \
date                                                            ...        
2021-10-01   100   100  100  100    0    0     0    0  100   0  ...    0   
2021-10-04   100   100  100  100    0    0     0    0  100   0  ...    0   
2021-10-05   100   100  100  100    0    0     0    0  100   0  ...    0   
2021-10-06   100   100  100  100    0    0     0    0  100   0  ...    0   
2021-10-07   100   100  100  100    0    0     0    0  100   0  ...    0   

            MSFT  NKE  PG  TRV  UNH  V  VZ  WBA  WMT  
date                                                  
2021-10-01     0    0   0  100  100  0   0    0    0  
2021-10-04     0    0   0  100  100  0   0    0    0  
2021-10-05     0    0   0  100  100  0   0    0    0  
2021-10-06     0  

In [31]:
#Backtesting with our pruned model
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all_tuned = backtest_stats(account_value=df_account_value_tuned)
perf_stats_all_tuned = pd.DataFrame(perf_stats_all_tuned)
perf_stats_all_tuned.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_tuned_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.023414
Cumulative returns     0.032951
Annual volatility      0.194674
Sharpe ratio           0.216339
Calmar ratio           0.131433
Stability              0.000626
Max drawdown          -0.178143
Omega ratio            1.037314
Sortino ratio          0.304229
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.914657
Daily value at risk   -0.024360
dtype: float64


In [32]:
#Now train with not tuned hyperaparameters
#Default config.ddpg_PARAMS
# non_tuned_model_ddpg = agent.get_model("ddpg",model_kwargs = config.DDPG_PARAMS )
# trained_ddpg = agent.train_model(model=non_tuned_model_ddpg, 
#                              tb_log_name='ddpg',
#                              total_timesteps=50000)
non_tuned_model_her = agent.get_model()
trained_her = agent.train_model(model=non_tuned_model_her, tb_log_name='her', total_timesteps=10000)

Using cuda device


In [33]:
df_account_value, df_actions = DRLHERAgent.DRL_prediction(
    model=trained_her, 
    environment = e_trade_gym)

hit end!


In [34]:
#Backtesting for not tuned hyperparamters
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.045797
Cumulative returns    -0.063557
Annual volatility      0.206524
Sharpe ratio          -0.124551
Calmar ratio          -0.175805
Stability              0.437430
Max drawdown          -0.260497
Omega ratio            0.979497
Sortino ratio         -0.172838
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.961662
Daily value at risk   -0.026122
dtype: float64


In [35]:
#You can see with trial, our sharpe ratio is increasing
#Certainly you can afford more number of trials for further optimization
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [36]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [37]:
#Hyperparamters importance
#Ent_coef is the most important
plot_param_importances(study)

## Further works

1.   You can tune more critical hyperparameters
2.   Multi-objective hyperparameter optimization using Optuna. Here we can maximize Sharpe and simultaneously minimize Volatility in our account value to tune our hyperparameters



In [38]:
plot_edf(study)